In [ ]:
import organelle_morphology

p = organelle_morphology.Project(
    "../data/cebra_em_example/seg_er_5nm_mito_10nm/", compression_level=2
)
# p.add_source("mito-it00_b0_6_stitched", organelle="mito")
p.add_source("er-it00_b0_5_stitched", organelle="er")
p.filter_organelles_by_size("er", 0.9)

In [ ]:
# Load mesh
# mesh_0 = p.organelles("er_9492")[0].mesh
mesh_1 = p.organelles("er_*")[1].mesh

In [ ]:
# packages used for moprhological analysis
from organelle_morphology.rendering import _map_curvature_to_colors
import numpy as np
from scipy.linalg import eig

In [ ]:
def compute_principal_curvatures_extended(
    mesh, vertex_index, vertex_normals, search_radius=2
):
    # Initialize a set for neighbors to ensure uniqueness
    # start with the closest neighbors vor each vertex
    extended_neighbors = set(mesh.vertex_neighbors[vertex_index])

    # an extended search radius can help with smoothing out the values a bit.
    if search_radius > 1:
        for _ in range(search_radius - 1):
            current_neighbors = list(extended_neighbors)
            for neighbor in current_neighbors:
                # Add neighbors of neighbors
                extended_neighbors.update(mesh.vertex_neighbors[neighbor])

    # Remove the original vertex if included
    extended_neighbors.discard(vertex_index)

    # Convert to list for indexing
    extended_neighbors = list(extended_neighbors)

    # Extract vertex normals of the extended neighbors
    neighbor_normals = vertex_normals[extended_neighbors]

    # Compute the covariance matrix of the normals
    covariance_matrix = np.cov(neighbor_normals, rowvar=False)

    # Compute eigenvalues (curvatures) and eigenvectors (directions)
    eigenvalues, eigenvectors = eig(covariance_matrix)

    # Sort eigenvalues in descending order
    idx = np.argsort(-eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # use the largest and smallest eigenvalues as principal curvatures
    k1 = eigenvalues[0]
    k2 = eigenvalues[-1]

    return k1, k2


def morph_analysis(mesh, search_radius=1):
    # Compute principal curvatures for each vertex
    k1_values = []
    k2_values = []
    for i in range(len(mesh.vertices)):
        k1, k2 = compute_principal_curvatures_extended(
            mesh, i, mesh.vertex_normals, search_radius=search_radius
        )
        k1_values.append(k1)
        k2_values.append(k2)

        # Convert to numpy arrays
    k1_values = np.array(k1_values)
    k2_values = np.array(k2_values)

    # Classify vertices based on principal curvatures
    curvature_difference = np.abs(k1_values - k2_values)

    # assign vertex color for visualization
    mesh.visual.vertex_colors = _map_curvature_to_colors(curvature_difference, "plasma")

    return mesh, curvature_difference

In [ ]:
mesh, curvature_difference = morph_analysis(mesh_1, search_radius=2)

In [ ]:
mesh.show()